# Calibracion estereo y computo de mapas de rectificacion

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import glob

In [ ]:
# Directorio base de los resultados de calibración
base_dir = "imgs/calibracion"

# Ruta al archivo existente de calibración estéreo
calib_file = os.path.join(base_dir, "stereo_calibration.pkl")

# Cargar la calibración
with open(calib_file, "rb") as f:
    calib = pickle.load(f)

# Imprimir los parámetros de calibración
def np_print(name, arr):
    np.set_printoptions(precision=4, suppress=True)
    print(f"\n{name} =\n{arr}")

print("### PARÁMETROS DE CALIBRACIÓN ESTÉREO ###")
np_print("Matriz intrínseca cámara izquierda (K1)", calib["left_K"])
np_print("Coef. distorsión izquierda (d1)", calib["left_dist"])
np_print("Matriz intrínseca cámara derecha (K2)", calib["right_K"])
np_print("Coef. distorsión derecha (d2)", calib["right_dist"])
np_print("Rotación (R)", calib["R"])
np_print("Traslación (T)", calib["T"])
np_print("Matriz esencial (E)", calib["E"])
np_print("Matriz fundamental (F)", calib["F"])

In [ ]:
# Extraer los parámetros necesarios
left_K = calib["left_K"]
left_dist = calib["left_dist"]
right_K = calib["right_K"]
right_dist = calib["right_dist"]
R = calib["R"]
T = calib["T"]

# Si pickle no tiene 'image_size', lo inferimos leyendo una imagen del dataset
if "image_size" in calib:
    image_size = calib["image_size"]
else:
    sample_img = cv2.imread(sorted(glob.glob(os.path.join(base_dir, "*left*.jpg"))[0]))
    image_size = (sample_img.shape[1], sample_img.shape[0])  # (w, h)
    print("⚠️ 'image_size' no estaba en el pickle, se infirió como:", image_size)

# Calcular la rectificación estéreo
R1, R2, P1, P2, Q, validRoi1, validRoi2 = cv2.stereoRectify(
    left_K, left_dist,
    right_K, right_dist,
    image_size,
    R, T,
    alpha=0
)

# Generar los mapas de rectificación
left_map_x, left_map_y = cv2.initUndistortRectifyMap(
    left_K, left_dist, R1, P1, image_size, cv2.CV_32FC1
)
right_map_x, right_map_y = cv2.initUndistortRectifyMap(
    right_K, right_dist, R2, P2, image_size, cv2.CV_32FC1
)

# Guardar los mapas en un nuevo pickle
stereo_maps = {
    "left_map_x": left_map_x,
    "left_map_y": left_map_y,
    "right_map_x": right_map_x,
    "right_map_y": right_map_y,
    "R1": R1,
    "R2": R2,
    "P1": P1,
    "P2": P2,
    "Q": Q,
    "validRoi1": validRoi1,
    "validRoi2": validRoi2,
}

maps_file = os.path.join(base_dir, "stereo_maps.pkl")
with open(maps_file, "wb") as f:
    pickle.dump(stereo_maps, f)

In [ ]:
# Mostrar ejemplo de par de imagenes des-distorsionadas y rectificadas

# Cargar los mapas de rectificación
with open("imgs/calibracion/stereo_maps.pkl", "rb") as f:
    maps = pickle.load(f)

left_map_x, left_map_y = maps["left_map_x"], maps["left_map_y"]
right_map_x, right_map_y = maps["right_map_x"], maps["right_map_y"]

# Tomar un par de imágenes de calibración
left_img_path = sorted(glob.glob("imgs/calibracion/*left*.jpg"))[0]
right_img_path = sorted(glob.glob("imgs/calibracion/*right*.jpg"))[0]

imgL = cv2.imread(left_img_path, cv2.IMREAD_GRAYSCALE)
imgR = cv2.imread(right_img_path, cv2.IMREAD_GRAYSCALE)

# Aplicar rectificación
rectL = cv2.remap(imgL, left_map_x, left_map_y, cv2.INTER_LINEAR)
rectR = cv2.remap(imgR, right_map_x, right_map_y, cv2.INTER_LINEAR)

# Mostrar
fig, axes = plt.subplots(1, 2, figsize=(12,6))
axes[0].imshow(rectL, cmap="gray")
axes[0].set_title("Izquierda rectificada")
axes[1].imshow(rectR, cmap="gray")
axes[1].set_title("Derecha rectificada")

# Dibujar líneas horizontales para verificar alineación epipolar
for y in [100, 300, 500]:
    axes[0].axhline(y, color='r', linestyle='--', linewidth=0.7)
    axes[1].axhline(y, color='r', linestyle='--', linewidth=0.7)

plt.show()

# Rectificacion, disparidad y profundidad

## Rectificar los pares de imagenes del dataset de reconstruccion

In [ ]:
def rectificar_imagenes(path_imgs, path_maps):
    """
    Rectifica todos los pares de imágenes estéreo (left/right) de un directorio usando mapas precomputados.
    
    Parámetros:
        path_imgs (str): Carpeta que contiene las imágenes 'left_*.jpg' y 'right_*.jpg'
        path_maps (str): Ruta al archivo 'stereo_maps.pkl'
    """
    # Crear carpeta de salida si no existe
    output_dir = os.path.join(path_imgs, "rectificadas")
    os.makedirs(output_dir, exist_ok=True)

    # Cargar los mapas de rectificación
    with open(path_maps, "rb") as f:
        maps = pickle.load(f)
    left_map_x, left_map_y = maps["left_map_x"], maps["left_map_y"]
    right_map_x, right_map_y = maps["right_map_x"], maps["right_map_y"]

    # Buscar las imágenes left y right
    left_imgs = sorted(glob.glob(os.path.join(path_imgs, "left_*.jpg")))
    right_imgs = sorted(glob.glob(os.path.join(path_imgs, "right_*.jpg")))

    if len(left_imgs) != len(right_imgs):
        print("⚠️ Cantidad distinta de imágenes left/right")
        print(f"Left: {len(left_imgs)}, Right: {len(right_imgs)}")
        return

    print(f"🔹 Rectificando {len(left_imgs)} pares de imágenes...")

    for left_path, right_path in zip(left_imgs, right_imgs):
        # Leer imágenes
        imgL = cv2.imread(left_path, cv2.IMREAD_GRAYSCALE)
        imgR = cv2.imread(right_path, cv2.IMREAD_GRAYSCALE)

        # Rectificar
        rectL = cv2.remap(imgL, left_map_x, left_map_y, cv2.INTER_LINEAR)
        rectR = cv2.remap(imgR, right_map_x, right_map_y, cv2.INTER_LINEAR)

        # Nombres de salida
        baseL = os.path.basename(left_path)
        baseR = os.path.basename(right_path)
        outL = os.path.join(output_dir, f"rect_{baseL}")
        outR = os.path.join(output_dir, f"rect_{baseR}")

        # Guardar
        cv2.imwrite(outL, rectL)
        cv2.imwrite(outR, rectR)

    print(f"✅ Imágenes rectificadas guardadas en: {output_dir}")

rectificar_imagenes("imgs/objeto", "imgs/calibracion/stereo_maps.pkl")

## Disparidad de la escena